In [ ]:
import os
import cv2
from ultralytics import YOLO
import csv
import joblib
import numpy as np
from datetime import datetime, timedelta
conf = 0.7
dir = "fall.v2i.multiclass"
dir2 = "train"
# Path to the directory containing images
input_dir = f'C:/Users/User/Desktop/dataset/{dir}/{dir2}'  # Directory containing images
output_dir = f'C:/Users/User/Desktop/dataset/{dir}/{dir2}_detect_{conf}'  # Output directory for annotated images
output_file = f"C:/Users/User/Desktop/dataset/{dir}/{dir2}_keypoints_{conf}.csv"  # Output CSV file for keypoints data

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load YOLOv8 model for pose detection
model = YOLO('yolov8n-pose.pt')  # Ensure you have the correct YOLOv8 pose model
m = "rf"
m_version = '3'
# my_model = tf.keras.models.load_model(f'new_{m}_model_{m_version}.h5')
my_model = joblib.load(f'new_{m}_model_{m_version}.pkl')
my_scaler = joblib.load(f'new_{m}_scaler_{m_version}.pkl')
def process_image(results):
    r = results[0]
    combined_results = []
    for i in range(len(r.keypoints.xyn)):
        for j in range(17):  # Assuming there are 17 keypoints
            x, y = r.keypoints.xyn[i][j].numpy()
            combined_results.append((x, y))
    box = results[0].boxes.xywhn.numpy().reshape(-1, 2)
    list_of_tuples = [tuple(row) for row in box]
    combined_results.extend(list_of_tuples)
    return combined_results
# Process each PNG image in the input directory
all_keypoints_data = []
# Get all files in input_dir
all_files = os.listdir(input_dir)
# Filter out only files that end with .png and are valid files
image_files = [filename for filename in all_files if os.path.isfile(os.path.join(input_dir, filename)) and filename.lower().endswith('.jpg')]
# Sort the filtered image files
image_files.sort()

for idx, filename in enumerate(image_files):
    input_path = os.path.join(input_dir, filename)
    
    # Output path remains the same as input path, with '_detect_confidence' appended
    output_path = os.path.join(output_dir, filename)

    # Load the image
    image = cv2.imread(input_path)

    # Perform pose detection
    results = model(image, conf=conf)
    
    if len(results[0].boxes) == 1: 
        r = results[0]
        for i in range(len(r.keypoints.xyn)):
            combined_results = []
            notice_time = datetime(1970, 1, 1)
            for j in range(17):  # Assuming there are 17 keypoints
                x, y = r.keypoints.xyn[i][j].numpy()
                combined_results.append((x, y))
            box = results[0].boxes[i].xywhn.numpy().reshape(-1, 2)
            list_of_tuples = [tuple(row) for row in box]
            combined_results.extend(list_of_tuples)
            data = np.array(combined_results).reshape(1,-1)
            data = my_scaler.transform(data)
            y_pred = my_model.predict(data)
        if y_pred[0] == 1:
            # Draw the results on the image
            annotated_image = results[0].plot()
            # Save the annotated image with the original filename
            cv2.imwrite(output_path, annotated_image)
            print(f"Processed and saved: {output_path}")
            keypoints_data = process_image(results)
            keypoints_data.insert(0, filename)  # Use the original filename
            keypoints_data = keypoints_data + [1]
            all_keypoints_data.append(keypoints_data)
        else:
            # Draw the results on the image
            annotated_image = results[0].plot()
            # Save the annotated image with the original filename
            cv2.imwrite(output_path, annotated_image)
            print(f"Processed and saved: {output_path}")
            keypoints_data = process_image(results)
            keypoints_data.insert(0, filename)  # Use the original filename
            keypoints_data = keypoints_data + [0]
            all_keypoints_data.append(keypoints_data)
    else:
        keypoints_data = [filename] + [None] * 19  # Assuming 17 keypoints with x,y pairs
        all_keypoints_data.append(keypoints_data)

# Write keypoints data to CSV
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write the header
    header = ["image_name"] + [f"x{i+1}, y{i+1}" for i in range(17)] + ["xb, yb"] + ["w, h"] + ["label"] # Assuming 17 keypoints with x,y pairs
    writer.writerow(header)
    
    # Write the data
    writer.writerows(all_keypoints_data)

print("Data saved to", output_file)



0: 640x640 (no detections), 145.9ms
Speed: 4.0ms preprocess, 145.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 133.9ms
Speed: 2.1ms preprocess, 133.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Processed and saved: C:/Users/User/Desktop/dataset/fall.v2i.multiclass/train_detect_0.7\0_Portrait-of-a-senior-man-indoor_jpg.rf.c02e6ba6a9c9e5cca989211aa1c34580.jpg

0: 640x640 1 person, 133.9ms
Speed: 2.0ms preprocess, 133.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Processed and saved: C:/Users/User/Desktop/dataset/fall.v2i.multiclass/train_detect_0.7\1-1-_jpg.rf.8b69f8375047b4e6d694b5195c579193.jpg

0: 640x640 1 person, 135.5ms
Speed: 2.0ms preprocess, 135.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Processed and saved: C:/Users/User/Desktop/dataset/fall.v2i.multiclass/train_detect_0.7\1-2-_jpg.rf.d066cb9bade27ba6c0c3b5327d2ac010.jpg

0: 640x640 1 person, 134.9ms
Speed: 2.1m